In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, IntSlider, FloatSlider
from IPython.display import display


data_folder = "../dataDec1"  
output_folder = "../processed_dataDec1"  
reference_file = os.path.join(data_folder, "Reference.d24")
sample_file = os.path.join(data_folder, "Sample.d24")

if not os.path.exists(reference_file):
    raise FileNotFoundError(f"Reference.d24 not found in: {reference_file}")
if not os.path.exists(sample_file):
    raise FileNotFoundError(f"Sample.d24 not found in: {sample_file}")


os.makedirs(output_folder, exist_ok=True)

def read_d24_file(file_name):
    with open(file_name, 'r') as file:
        lines = file.readlines()
    start_line = next(i for i, line in enumerate(lines) if line.lstrip()[0].isdigit())
    data_lines = lines[start_line:]
    return pd.DataFrame(
        [list(map(float, line.split())) for line in data_lines],
        columns=["Time (ps)", "Signal", "Column3", "Column4", "Column5"]
    )[["Time (ps)", "Signal"]]

# Load data
reference_data = read_d24_file(reference_file)
sample_data = read_d24_file(sample_file)


processed_reference = None
processed_sample = None

def plot_with_hann_and_fft(ref_window_width, ref_mult_const, sample_window_width, sample_mult_const):
    global processed_reference, processed_sample
    
    # Generate Hann windows and scale them by the multiplication constants
    ref_hann_window = ref_mult_const * np.hanning(ref_window_width)
    sample_hann_window = sample_mult_const * np.hanning(sample_window_width)
    
    def apply_hann(data, hann_window):
        signal = data["Signal"].values
        padded_window = np.pad(hann_window, (0, len(signal) - len(hann_window)), 'constant')
        return signal * padded_window

    # Apply Hann windows
    reference_windowed = apply_hann(reference_data, ref_hann_window)
    sample_windowed = apply_hann(sample_data, sample_hann_window)

    # Store processed data for saving
    processed_reference = reference_data.copy()
    processed_sample = sample_data.copy()
    processed_reference["Signal"] = reference_windowed
    processed_sample["Signal"] = sample_windowed

    # Compute FFT
    ref_fft = np.abs(np.fft.fft(reference_windowed))
    sample_fft = np.abs(np.fft.fft(sample_windowed))
    freqs = np.fft.fftfreq(len(reference_windowed), d=(reference_data["Time (ps)"].iloc[1] - reference_data["Time (ps)"].iloc[0]))

    # Plot
    fig, axs = plt.subplots(2, 1, figsize=(12, 10))

    # Time-domain plots
    axs[0].plot(reference_data["Time (ps)"], reference_data["Signal"], label="Reference Original", color='blue')
    axs[0].plot(reference_data["Time (ps)"], reference_windowed, label="Reference Windowed", linestyle="--", color='cyan')
    axs[0].plot(sample_data["Time (ps)"], sample_data["Signal"], label="Sample Original", color='red')
    axs[0].plot(sample_data["Time (ps)"], sample_windowed, label="Sample Windowed", linestyle="--", color='orange')
    axs[0].set_title(f"Time-Domain Signals with Hann Window\n(Ref: {ref_window_width}, Sample: {sample_window_width}, Ref Const: {ref_mult_const:.2f}, Sample Const: {sample_mult_const:.2f})")
    axs[0].set_xlabel("Time (ps)")
    axs[0].set_ylabel("Signal")
    axs[0].legend()
    axs[0].grid(True)

    # Frequency-domain (FFT) plots
    axs[1].plot(freqs[:len(freqs)//2], ref_fft[:len(freqs)//2], label="Reference FFT", color='blue')
    axs[1].plot(freqs[:len(freqs)//2], sample_fft[:len(freqs)//2], label="Sample FFT", color='red')
    axs[1].set_title("Frequency-Domain (Fourier Transform)")
    axs[1].set_xlabel("Frequency (THz)")
    axs[1].set_ylabel("Amplitude")
    axs[1].legend()
    axs[1].grid(True)

    plt.tight_layout()
    plt.show()

# Create sliders for the two Hann windows, multiplication constants, and interactive plot
interact(
    plot_with_hann_and_fft, 
    ref_window_width=IntSlider(min=10, max=500, step=10, value=230, description="Ref Width"),
    ref_mult_const=FloatSlider(min=0.1, max=2.0, step=0.1, value=1.0, description="Ref Const"),
    sample_window_width=IntSlider(min=10, max=500, step=10, value=220, description="Sample Width"),
    sample_mult_const=FloatSlider(min=0.1, max=2.0, step=0.1, value=1.0, description="Sample Const")
);


# Save processed files when the user is done
def save_processed_files():
    if processed_reference is not None and processed_sample is not None:
        ref_output_file = os.path.join(output_folder, "Processed_Reference.d24")
        sample_output_file = os.path.join(output_folder, "Processed_Sample.d24")
        
        # Save processed Reference
        processed_reference.to_csv(ref_output_file, sep="\t", index=False, header=False)
        print(f"Processed Reference saved to {ref_output_file}")

        # Save processed Sample
        processed_sample.to_csv(sample_output_file, sep="\t", index=False, header=False)
        print(f"Processed Sample saved to {sample_output_file}")
    else:
        print("No processed data to save.")

# Display a button to save files
from ipywidgets import Button
save_button = Button(description="Save Processed Files")
save_button.on_click(lambda x: save_processed_files())
display(save_button)


interactive(children=(IntSlider(value=230, description='Ref Width', max=500, min=10, step=10), FloatSlider(val…

Button(description='Save Processed Files', style=ButtonStyle())

Processed Reference saved to ../processed_dataDec1\Processed_Reference.d24
Processed Sample saved to ../processed_dataDec1\Processed_Sample.d24
